In [8]:
import tensorflow as tf
import numpy as np

## log_loss

In [4]:
l = tf.placeholder(tf.int64, shape=[3])
p = tf.placeholder(tf.int64, shape=[3])
ll = tf.losses.log_loss(labels=l, predictions=p)

In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
print(sess.run(ll, feed_dict={l: [1, 2, 3], p: [1, 2, 3]}))

nan


## numpy.random.choice

In [17]:
a = [1, 2, 3]
p = [0.1, 0.1, 0.8]
r = np.random.choice(a, size=100000, p=p)
s1 = np.sum(r == 1)
s2 = np.sum(r == 2)
s3 = np.sum(r == 3)
d = s1 + s2 + s3
print(s1/d, s2/d, s3/d)

0.10181 0.09952 0.79867
